## Hay que hacer una autenticacion al principio

In [17]:
#desautenticando:
import ee
ee.Reset()  # limpia cualquier sesión previa


In [18]:
import ee
ee.Authenticate(
    authorization_code=None,
    scopes=[
        'https://www.googleapis.com/auth/earthengine',
        'https://www.googleapis.com/auth/devstorage.full_control',
        'https://www.googleapis.com/auth/drive'
    ],
    quiet=False,
    auth_mode='notebook'
)
ee.Initialize()


In [19]:
print(ee.Image("ECMWF/ERA5_LAND/DAILY_AGGR/20210101").bandNames().getInfo())


['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_open_water_surfaces_excluding_oceans_sum', '

## Verificando que se me haya dado acceso

In [20]:
import ee

ee.Initialize()

# Probar accediendo a un dataset simple
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').first()
print(dataset.getInfo())


{'type': 'Image', 'bands': [{'id': 'dewpoint_temperature_2m', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3601, 1801], 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}, {'id': 'temperature_2m', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3601, 1801], 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}, {'id': 'skin_temperature', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3601, 1801], 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}, {'id': 'soil_temperature_level_1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3601, 1801], 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}, {'id': 'soil_temperature_level_2', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3601, 1801], 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}, {'id': 'soil_tem

In [23]:
# --- Instalaciones (solo si no las tenés) ---
# pip install earthengine-api geemap pandas

import ee
import pandas as pd

# Inicializar Earth Engine
ee.Initialize()

# --- Coordenadas de Bahía Blanca ---
bahiablanca = ee.Geometry.Point([-62.2683, -38.7196])

# --- Variables que queremos extraer ---
variables = [
    # 🌧️ Precipitación
    'total_precipitation_sum',
    'snowmelt_sum',
    'snowfall_sum',

    # 🌊 Escorrentía y acumulación de agua
    'runoff_sum',
    'surface_runoff_sum',
    'sub_surface_runoff_sum',
    'total_evaporation_sum',

    # 💧 Contenido de agua en el suelo
    'volumetric_soil_water_layer_1',
    'volumetric_soil_water_layer_2',
    'soil_temperature_level_1',

    # 🌤️ Radiación y evaporación
    'surface_net_solar_radiation_sum',
    'surface_thermal_radiation_downwards_sum',
    'surface_latent_heat_flux_sum',

    # 🌡️ Temperatura del aire y superficie
    'temperature_2m',
    'skin_temperature',
    'soil_temperature_level_1',
    'temperature_2m_min',
    'temperature_2m_max',

    # 🌬️ Viento y presión (ya estaban)
    'dewpoint_temperature_2m',
    'surface_solar_radiation_downwards_sum',
    'surface_pressure',
    'u_component_of_wind_10m',
    'v_component_of_wind_10m'
]


# --- Colección ERA5-Land ---
dataset = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .filterDate('2020-01-01', '2025-11-11')
    .select(variables)
)

# --- Función para obtener valores diarios en el punto ---
def extract_daily_values(image):
    mean_dict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bahiablanca,
        scale=1000
    )
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    feature = ee.Feature(None, mean_dict.set('date', date))
    return feature

# Aplicamos la función a cada imagen de la colección
features = dataset.map(extract_daily_values)

# --- Convertir el FeatureCollection a una lista de diccionarios ---
data = features.getInfo()['features']

# --- Pasar a pandas DataFrame ---
rows = []
for f in data:
    props = f['properties']
    props['localidad'] = 'Bahía Blanca'
    props['lat_lon'] = '-38.7196, -62.2683'
    rows.append(props)

df = pd.DataFrame(rows)

# --- Guardar como CSV ---
df.to_csv('bahia_blanca_era5.csv', index=False, encoding='utf-8')

print("✅ CSV creado con éxito: bahia_blanca_era5.csv")
print(df.head())


✅ CSV creado con éxito: bahia_blanca_era5.csv
         date  dewpoint_temperature_2m  surface_pressure  \
0  2020-01-01               288.206130     101471.793457   
1  2020-01-02               286.899150     101507.346354   
2  2020-01-03               287.632617     101440.541341   
3  2020-01-04               288.807709     100873.345540   
4  2020-01-05               288.468564     100257.991862   

   surface_solar_radiation_downwards_sum  temperature_2m  \
0                             27459336.0      293.423794   
1                             32500472.0      294.960049   
2                             32071140.0      298.117014   
3                             18240432.0      297.227221   
4                             14994120.0      293.302716   

   u_component_of_wind_10m  v_component_of_wind_10m     localidad  \
0                -4.374050                 2.201365  Bahía Blanca   
1                -2.676633                -2.323653  Bahía Blanca   
2                -0.38376